#### Scenario 1 : Design a python solution to extract data from an API endpoint

In [ ]:
import requests
import pandas as pd

API_KEY = '8f6ecfca64d58d9637026b7a63f9dbc4'
url = f"https://api.weatherstack.com/current?access_key={API_KEY}"

class WeatherStack:
  def __init__(self):
    self.df_weather_stack = pd.DataFrame()
  def get_weather_stack_data(self,query):
    response = requests.get(url, params={'query':query})
    result = response.json()
    if 'request' in result.keys():
      new_row = {**result.get('request') , **result.get('location') , **result.get('current')}
      self.df_weather_stack = pd.concat([self.df_weather_stack, pd.DataFrame([new_row])], ignore_index=True)
    return self.df_weather_stack


query_list = ["New Delhi","Tamil Nadu", "Karnataka", "Rajasthan", "West Bengal","Maharashtra"]

ws = WeatherStack()

for query in query_list:
  ws.get_weather_stack_data(query)


ws.df_weather_stack




,type,query,language,unit,name,country,region,lat,lon,timezone_id,...,wind_degree,wind_dir,pressure,precip,humidity,cloudcover,feelslike,uv_index,visibility,is_day
0,City,"New Delhi, India",en,m,New Delhi,India,Delhi,28.600,77.200,Asia/Kolkata,...,292,WNW,1016,0,35,50,22,5,4,yes
1,City,"Chennai, India",en,m,Chennai,India,Tamil Nadu,13.083,80.283,Asia/Kolkata,...,75,ENE,1013,0,66,25,34,9,5,yes
2,City,"Maisuru, India",en,m,Maisuru,India,Karnataka,12.307,76.650,Asia/Kolkata,...,78,ENE,1012,0,27,3,29,11,10,yes
3,City,"Bhilwara, India",en,m,Bhilwara,India,Rajasthan,25.350,74.633,Asia/Kolkata,...,155,SSE,1014,0,15,1,25,6,10,yes
4,City,"Kolkata, India",en,m,Kolkata,India,West Bengal,22.570,88.370,Asia/Kolkata,...,288,WNW,1014,0,37,0,25,6,4,yes
5,City,"Mumbai, India",en,m,Mumbai,India,Maharashtra,18.975,72.826,Asia/Kolkata,...,319,NW,1012,0,32,0,38,7,3,yes


#### Scenario 2 : Design and Implement Webserver Log Parser and Calculate reporting metrics

In [83]:
import requests
import re
import pandas as pd

# URL of the raw data file
url = 'https://raw.githubusercontent.com/elastic/examples/refs/heads/master/Common%20Data%20Formats/apache_logs/apache_logs'


class WebserverLogParser:
  def __init__(self):
    self.logs_parsed_data = pd.DataFrame()
  def get_logs(self):
    response = requests.get(url)
    if response.status_code == 200:
      logs = response.text.split('\n')
      return logs
    return None
  def parse_logs(self,logs):
    pattern = r'(?P<ip>\d+\.\d+\.\d+\.\d+) - - \[(?P<timestamp>.*?)\] "(?P<method>\S+) (?P<url>.*?) HTTP/\S+" (?P<status_code>\d+) \d+ "(?P<referrer>.*?)" "(?P<user_agent>.*?)"'
    # Regex patterns for extracting OS and browser from the User-Agent string
    os_pattern = r'\((.*?)\)'
    browser_pattern = r"(\w+)\/[\d\.]+"
    for log in logs:
      # Match the log entry using the pattern
      match = re.match(pattern, log)

      if match:
          log_data = match.groupdict()

          # Extract OS from User-Agent string
          os_match = re.search(os_pattern, log_data['user_agent'])
          os_info = os_match.group(1) if os_match else "Unknown OS"

          # Find all matches
          matches = re.findall(browser_pattern, log_data['user_agent'])
          browser_info = None
          if len(matches)==1:
            browser_info = matches[-1]
          elif len(matches)>1:
            browser_info = matches[-2]
          else:
            browser_info = "Unknown Browser"




          log_data['os'] = os_info
          log_data['browser'] = browser_info

          self.logs_parsed_data = pd.concat([self.logs_parsed_data, pd.DataFrame([log_data])], ignore_index=True)





wlp = WebserverLogParser()
logs = wlp.get_logs()
wlp.parse_logs(logs)
logs_df = wlp.logs_parsed_data
wlp.logs_parsed_data.head(8)

,ip,timestamp,method,url,status_code,referrer,user_agent,os,browser
0,83.149.9.216,17/May/2015:10:05:03 +0000,GET,/presentations/logstash-monitorama-2013/images...,200,http://semicomplete.com/presentations/logstash...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...,Macintosh; Intel Mac OS X 10_9_1,Chrome
1,83.149.9.216,17/May/2015:10:05:43 +0000,GET,/presentations/logstash-monitorama-2013/images...,200,http://semicomplete.com/presentations/logstash...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...,Macintosh; Intel Mac OS X 10_9_1,Chrome
2,83.149.9.216,17/May/2015:10:05:47 +0000,GET,/presentations/logstash-monitorama-2013/plugin...,200,http://semicomplete.com/presentations/logstash...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...,Macintosh; Intel Mac OS X 10_9_1,Chrome
3,83.149.9.216,17/May/2015:10:05:12 +0000,GET,/presentations/logstash-monitorama-2013/plugin...,200,http://semicomplete.com/presentations/logstash...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...,Macintosh; Intel Mac OS X 10_9_1,Chrome
4,83.149.9.216,17/May/2015:10:05:07 +0000,GET,/presentations/logstash-monitorama-2013/plugin...,200,http://semicomplete.com/presentations/logstash...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...,Macintosh; Intel Mac OS X 10_9_1,Chrome
5,83.149.9.216,17/May/2015:10:05:34 +0000,GET,/presentations/logstash-monitorama-2013/images...,200,http://semicomplete.com/presentations/logstash...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...,Macintosh; Intel Mac OS X 10_9_1,Chrome
6,83.149.9.216,17/May/2015:10:05:57 +0000,GET,/presentations/logstash-monitorama-2013/css/fo...,200,http://semicomplete.com/presentations/logstash...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...,Macintosh; Intel Mac OS X 10_9_1,Chrome
7,83.149.9.216,17/May/2015:10:05:50 +0000,GET,/presentations/logstash-monitorama-2013/css/fo...,200,http://semicomplete.com/presentations/logstash...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...,Macintosh; Intel Mac OS X 10_9_1,Chrome


In [32]:
# 1. Total Requests
logs_df.shape[0]

9330

In [40]:
# 2. Number of unique IP Addresses.
logs_df['ip'].nunique()

1674

In [49]:
# 3. TOP 10 most frequent IP addresses.
logs_df.groupby('ip').agg(count=('ip', 'size')).sort_values(by='count',ascending=False)[:10]


,count
ip,
66.249.73.135,432
46.105.14.53,364
130.237.218.86,293
50.16.19.13,113
209.85.238.199,102
75.97.9.59,99
68.180.224.225,95
198.46.149.143,82
208.115.111.72,73


In [50]:
# 4.Top 10 most requested URL paths.
logs_df.groupby('url').agg(count=('url', 'size')).sort_values(by='count',ascending=False)[:10]

,count
url,
/favicon.ico,788
/style2.css,532
/reset.css,528
/images/jordan-80.png,522
/images/web/2009/banner.png,506
/blog/tags/puppet?flav=rss20,488
/projects/xdotool/,221
/?flav=rss20,217
/,194


In [82]:
# 5. Busiest hour of the day(based on number of requests)

timestamps = pd.to_datetime(logs_df['timestamp'].values, format="%d/%b/%Y:%H:%M:%S %z")
logs_df['timestamp_hour'] = timestamps
logs_df['timestamp_hour']  = logs_df['timestamp_hour'].apply(lambda x: x.strftime('%Y-%m-%d %H:00:00'))
logs_df['timestamp_hour'].value_counts()[:5]

,count
timestamp_hour,
2015-05-19 19:00:00,131
2015-05-19 14:00:00,129
2015-05-18 15:00:00,127
2015-05-20 03:00:00,126
2015-05-17 20:00:00,125
